In [317]:
import numpy as np
import pandas as pd
import re

In [318]:
df = pd.read_excel("../data/Dailylog data -lifelog/Lifestyle - Sleep.xlsx")

In [319]:
df['Bedtime'] = pd.to_datetime(df['Bedtime'], format = '%Y-%m-%d %H:%M:%S')
df['Wake up time'] = pd.to_datetime(df['Wake up time'], format = '%Y-%m-%d %H:%M:%S')

In [320]:
df.rename(columns={"Non-identifying keys":"id","Measurement (-1: no value, 0: unknown sleep, 1: wake, 2: light sleep, 3: light sleep, 4: deep sleep)":"Measurement"},inplace=True)

In [321]:
df.isna().sum()

id                   0
Date                 0
Measurement types    0
Units of measure     0
Measurement          0
Bedtime              0
Wake up time         0
dtype: int64

In [322]:
df["Measurement"] = df["Measurement"].apply(lambda x: list(map(int, re.findall(r'\d', x))))

In [323]:
df_exploded=df.explode("Measurement")

In [324]:
sleep_patterns=pd.pivot_table(
    df_exploded,
    index=["id","Date"],
    columns="Measurement",
    aggfunc="size",
    fill_value=0
).reset_index()
cols = {-1: "no value", 0: "unknown sleep", 1: "wake", 2: "light sleep_1", 3: "light sleep_2", 4: "deep sleep"}
sleep_patterns.rename(columns=cols,inplace=True)

In [325]:
sleep_patterns.drop(columns=9,inplace=True)

In [326]:
df=pd.merge(df,sleep_patterns,on=["id","Date"])

In [327]:
df["sleep_duration"]=df["Wake up time"]-df["Bedtime"]

In [328]:
df["total_sleep_time"]=df["unknown sleep"]+df["light sleep_1"]+df["light sleep_2"]+df["deep sleep"]

In [329]:
df["sleep_efficiency"]=(df["total_sleep_time"]/df["sleep_duration"].dt.total_seconds())*100

In [330]:
df["sleep_duration"] = df["sleep_duration"].dt.total_seconds()/3600

In [331]:
df["sleep_in"]=df["Date"]-df["Bedtime"]
df["sleep_out"]=df["Date"]-df["Wake up time"]

df["sleep_in"]=df["sleep_in"].dt.total_seconds()/3600
df["sleep_out"]=df["sleep_out"].dt.total_seconds()/3600

In [332]:
df.drop(columns=["Measurement","Units of measure","Measurement types","total_sleep_time","Bedtime","Wake up time"],inplace=True)

In [333]:
df.to_csv("../data_processed/daily_lifelog/sleep",index=False)